In [7]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [8]:
def prepare_data(original_dataset_dir, base_dir):
    os.makedirs(base_dir, exist_ok=True)

    train_dir = os.path.join(base_dir, 'train')
    os.makedirs(train_dir, exist_ok=True)
    validation_dir = os.path.join(base_dir, 'validation')
    os.makedirs(validation_dir, exist_ok=True)

    train_cats_dir = os.path.join(train_dir, 'cats')
    os.makedirs(train_cats_dir, exist_ok=True)
    train_dogs_dir = os.path.join(train_dir, 'dogs')
    os.makedirs(train_dogs_dir, exist_ok=True)

    validation_cats_dir = os.path.join(validation_dir, 'cats')
    os.makedirs(validation_cats_dir, exist_ok=True)
    validation_dogs_dir = os.path.join(validation_dir, 'dogs')
    os.makedirs(validation_dogs_dir, exist_ok=True)

    cat_fnames = [fname for fname in os.listdir(original_dataset_dir) if fname.startswith('cat')]
    dog_fnames = [fname for fname in os.listdir(original_dataset_dir) if fname.startswith('dog')]

    train_cats, validation_cats = train_test_split(cat_fnames, test_size=0.2,train_size=0.8,random_state=42)
    train_dogs, validation_dogs = train_test_split(dog_fnames, test_size=0.2,train_size=0.8,random_state=42)

    for fname in train_cats:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_cats_dir, fname)
        shutil.copyfile(src, dst)

    for fname in validation_cats:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(validation_cats_dir, fname)
        shutil.copyfile(src, dst)

    for fname in train_dogs:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_dogs_dir, fname)
        shutil.copyfile(src, dst)

    for fname in validation_dogs:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(validation_dogs_dir, fname)
        shutil.copyfile(src, dst)

    return train_dir, validation_dir

In [9]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
def train_model(model, train_dir, validation_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

    validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=25,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

In [11]:
def classify_image(model, image_path):
    test_image = load_img(image_path, target_size=(64, 64))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)

    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    return prediction

In [12]:
if __name__ == "__main__":
    original_dataset_dir = 'train/train'
    base_dir = 'train/train'
    train_dir, validation_dir = prepare_data(original_dataset_dir, base_dir)
    model = define_model()
    train_model(model, train_dir, validation_dir)
    


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/25
625/625 [==============================] - 261s 415ms/step - loss: 0.6196 - accuracy: 0.6518 - val_loss: 0.5629 - val_accuracy: 0.6980
Epoch 2/25
625/625 [==============================] - 105s 167ms/step - loss: 0.5371 - accuracy: 0.7332 - val_loss: 0.5425 - val_accuracy: 0.7324
Epoch 3/25
625/625 [==============================] - 103s 164ms/step - loss: 0.4987 - accuracy: 0.7511 - val_loss: 0.4564 - val_accuracy: 0.7804
Epoch 4/25
625/625 [==============================] - 109s 174ms/step - loss: 0.4670 - accuracy: 0.7742 - val_loss: 0.5007 - val_accuracy: 0.7546
Epoch 5/25
625/625 [==============================] - 103s 165ms/step - loss: 0.4455 - accuracy: 0.7888 - val_loss: 0.4789 - val_accuracy: 0.7798
Epoch 6/25
625/625 [==============================] - 100s 160ms/step - loss: 0.4279 - accuracy: 0.7991 - val_loss: 0.4200 - val_accuracy: 0.8018
Epoch 7/25
625/625 [===================

In [13]:
from sklearn.metrics import confusion_matrix, classification_report
def evaluate_model(model, validation_dir):
    test_datagen = ImageDataGenerator(rescale=1./255)
    validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary',
        shuffle=False)  
    true_labels = validation_generator.classes

    predictions = model.predict(validation_generator, steps=len(validation_generator))
    predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]

    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    print("Confusion Matrix:")
    print(conf_matrix)

    class_report = classification_report(true_labels, predicted_labels, target_names=["cat", "dog"])
    print("\nClassification Report:")
    print(class_report)

evaluate_model(model, validation_dir)

Found 5000 images belonging to 2 classes.
157/157 [==============================] - 11s 65ms/step
Confusion Matrix:
[[1945  555]
 [ 310 2190]]

Classification Report:
              precision    recall  f1-score   support

         cat       0.86      0.78      0.82      2500
         dog       0.80      0.88      0.84      2500

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.83      0.83      0.83      5000



In [16]:
image_path = 'test1/test1/12471.jpg'
print(f"The image is of a {classify_image(model, image_path)}")

1/1 [==============================] - 0s 108ms/step
The image is of a dog
